In [12]:
import json
import pandas as pd
import matplotlib as plt
import re

In [3]:
for i in range(2,5):
    data = 'data/data' + str(i) + '.txt'

    tweets_data = []
    tweets_file = open(data, "r")
    for line in tweets_file:
        try:
            tweet = json.loads(line)
            tweets_data.append(tweet)
        except:
            continue
            
len(tweets_data)

54437

In [11]:
data = pd.DataFrame(tweets_data)

(54437, 32)

In [15]:
def processTweet(tweet):
    #Convert to lower case
    tweet = tweet.lower()
    #Convert www.* or https?://* to URL
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',tweet)
    #Convert @username to AT_USER
    tweet = re.sub('@[^\s]+','AT_USER',tweet)
    #Remove additional white spaces
    tweet = re.sub('[\s]+', ' ', tweet)
    #Replace #word with word
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    #trim
    tweet = tweet.strip('\'"')
    return tweet

In [17]:
for tweet in data['text']:
    

u'rt AT_USER reasons to hate donald trump URL'